In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
import gensim
import re, string
from scipy.sparse import csr_matrix, hstack
import sklearn
from sklearn import feature_extraction
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import stop_words
from sklearn.preprocessing import PolynomialFeatures

%matplotlib inline

In [ ]:
hours = pd.read_excel('ActivitiesGangHours.xlsx', index = 'Activity ID')
hours.drop('Activities Description', axis=1, inplace=True) #TODO: revise!!!

In [ ]:
df = pd.read_excel('ActivityConstrains.xlsx', index = 'Activity ID')
df.drop(['Name','Unnamed: 7'], axis=1,inplace=True)
df = df.dropna()###disabel if instructions how to handle provided
df["Activity ID"]=df["Activity ID"].astype(int)

In [ ]:
df.columns = ['ID','Description','Element','Extractor','Operation','Value','Unit']

In [ ]:
df.loc[df.Unit=='mm','Value']/=1000
df.loc[df.Unit=='mm','Unit']='m'

### Values and categories processing

In [ ]:
cat_df = df[df.Operation!='=']
cat_df['cat_value'] = cat_df[['Element','Extractor','Value']].groupby(by=['Element','Extractor'])['Value'].transform(lambda x: pd.factorize(x, sort=True)[0])
cat_df['cat_value']+=1 ##To leave zeros for non-interval values
cat_df.loc[cat_df['Operation'].isin(['>','>=','=>']),'cat_value']+=1

In [ ]:
df['cat_value']=0
df.loc[cat_df.index,'cat_value']=cat_df['cat_value']
df.loc[cat_df.index,'Value']=0
df.Value +=df.cat_value

In [ ]:
df['Op']=0
df.loc[df.Operation=='=','Op']=1
df['ElExtOp']=df.Element +  df.Extractor + df.Op.astype(str)

In [ ]:
sp = df.pivot(columns='ElExtOp', values='Value').fillna(0.0)
sp['ID']=df.ID
sp = sp.groupby(by='ID').sum().reset_index()

### Description processing

In [ ]:
### extract numbers here

In [ ]:
regex = re.compile('[' +re.escape(string.punctuation) + '\d\.\\r\\t\\n]')
descr = df['Description'].apply(lambda x: regex.sub(" ",x))

In [ ]:
stopwords = df.Extractor.unique().tolist()
stopwords += df.Element.unique().tolist()
stopwords += df.Unit.unique().tolist()
stopwords += stop_words.ENGLISH_STOP_WORDS
stopwords += ['exceeding','over','thickness', 'size','like','mm']
len(stopwords)

In [ ]:
vectorizer = TfidfVectorizer(analyzer='word',stop_words=frozenset(stopwords))
vectorizer.fit(descr)

In [ ]:
descr_df = pd.DataFrame(vectorizer.transform(descr).todense(),columns=list(vectorizer.vocabulary_.keys())).fillna(0.0)

In [ ]:
descr_df.index = df.index
descr_df['ID']=df.ID.astype(int)
descr_df = descr_df.groupby(by='ID').mean().reset_index()

### Merging and getting polynomial features

In [ ]:
ds = descr_df.merge(sp,on='ID',how='left')

In [ ]:
ds.to_csv('table.csv', index=False) ###To be able 

In [ ]:
ds = pd.read_csv('table.csv')

In [ ]:
ds=pd.SparseDataFrame(ds)

In [ ]:
pf = PolynomialFeatures(2,True,False)
pf.fit(ds.drop('ID',axis=1))

In [ ]:
dsp = pf.transform(ds.drop('ID',axis=1))

In [ ]:
dsp_df = pd.SparseDataFrame(dsp,columns=pf.get_feature_names())

In [ ]:
dsp_df.shape

In [ ]:
dsp_df.to_csv('poly.csv',index=False)

In [ ]:
vectorizer = CountVectorizer(min_df=0, token_pattern=r"\b\w+\b")
vectorizer.fit(df.Unit.unique())
uns = vectorizer.transform(df.Unit)

index = [i for i in range(1, uns.shape[0]+1)]
units = pd.SparseDataFrame(uns, index=index, columns=vectorizer.get_feature_names())
units.fillna(0, inplace=True)